In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import data_cleaning as dc
import pickle
import os

In [8]:
vista_raw_path = '/Users/noahdimonti/Documents/Uni/MPhil in Eng and CS/EV Apartment Building Project/Modelling Data/Potential Datasets/vista/T_VISTA1218_V1.csv'
work_ev_path =  '/Users/noahdimonti/Documents/Uni/MPhil in Eng and CS/EV Apartment Building Project/Modelling Data/Potential Datasets/vista/VISTA_Time_HomeDeAr_WorkVehicle.csv'
casual_ev_path = '/Users/noahdimonti/Documents/Uni/MPhil in Eng and CS/EV Apartment Building Project/Modelling Data/Potential Datasets/vista/VISTA_Time_HomeDeAr_CasualVehicle.csv'
ev_model_path = '/Users/noahdimonti/Developer/PycharmProjects/EV_model_test/'

In [3]:
casual_ev = pd.read_csv(casual_ev_path)
casual_ev.rename(columns={'Three _2nd_ARRTIME': 'Three_2nd_ARRTIME',
                          'Three _2nd_DEPTIME': 'Three_2nd_DEPTIME'}, inplace=True)
casual_ev = dc.clean_raw_data(casual_ev, casual_ev.columns)
casual_ev = dc.convert_to_timestamp(casual_ev, casual_ev.columns)
casual_ev

In [11]:
casual_ev_once = casual_ev[['Once_DEPTIME', 'Once_ARRTIME']]
casual_ev_twice = casual_ev[['Twice_1st_DEPTIME', 'Twice_1st_ARRTIME', 'Twice_2nd_DEPTIME', 'Twice_2nd_ARRTIME']]
casual_ev_three = casual_ev[['Three_1st_DEPTIME', 'Three_1st_ARRTIME', 'Three_2nd_DEPTIME', 'Three_2nd_ARRTIME', 'Three_3rd_DEPTIME', 'Three_3rd_ARRTIME']]

casual_ev_once = dc.remove_outliers_once(casual_ev_once, 'Once_DEPTIME', 'Once_ARRTIME')
casual_ev_twice = dc.remove_outliers_twice(casual_ev_twice)
casual_ev_three = dc.remove_outliers_three(casual_ev_three)

In [9]:
work_ev = pd.read_csv(work_ev_path)
work_ev = dc.clean_raw_data(work_ev, work_ev.columns)
work_ev = dc.convert_to_timestamp(work_ev, work_ev.columns)
work_ev = dc.remove_outliers_once(work_ev, 'DEPTIME', 'ARRTIME')
work_ev

,DEPTIME,ARRTIME
0,2019-01-01 08:45:00,2019-01-01 17:45:00
1,2019-01-01 05:30:00,2019-01-01 23:00:00
2,2019-01-01 11:45:00,2019-01-01 16:15:00
3,2019-01-01 07:00:00,2019-01-01 22:00:00
4,2019-01-01 07:30:00,2019-01-01 17:45:00
...,...,...
10500,2019-01-01 08:30:00,2019-01-01 17:30:00
10501,2019-01-01 08:30:00,2019-01-01 18:45:00
10502,2019-01-01 07:00:00,2019-01-01 16:30:00
10503,2019-01-01 07:00:00,2019-01-01 16:30:00


In [11]:
class EVData:
    def __init__(self):
        self.t_arr = []
        self.timeseries = None


def create_timeseries_at_home_pattern_once(df, dep_col: str, arr_col: str, time_resolution=15, num_of_ev=10,
                                           start_date_time='2019-01-01 00:00:00', rand_state=0):

    sample = df.sample(num_of_ev, random_state=rand_state)

    ev_data = []
    for sample_row in sample.iterrows():
        timeseries = pd.DataFrame(pd.date_range(start=start_date_time, periods=(60/time_resolution)*24,
                                                freq=f'{time_resolution}min'),

                                  columns=['timestamp'])
        home_status = []
        data = EVData()

        for timeseries_row in timeseries.iterrows():
            if timeseries_row[1].timestamp < sample_row[1][f'{dep_col}']:
                home_status.append(1)
            elif ((timeseries_row[1].timestamp >= sample_row[1][f'{dep_col}']) &
                  (timeseries_row[1].timestamp < sample_row[1][f'{arr_col}'])):
                home_status.append(0)
            elif timeseries_row[1].timestamp >= sample_row[1][f'{arr_col}']:
                home_status.append(1)

        data.t_arr.append(str(sample_row[1][f'{arr_col}']))

        timeseries[f'EV_ID{sample_row[0]}'] = home_status
        data.timeseries = timeseries

        ev_data.append(data)

    return ev_data, sample

In [12]:
data, sample = create_timeseries_at_home_pattern_once(work_ev, 'DEPTIME', 'ARRTIME', num_of_ev=2)
data[0].t_arr, data[1].t_arr, sample, data

/var/folders/5g/f459qk8s5ql55568q_c08sxr0000gn/T/ipykernel_16325/3678814824.py:14: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  timeseries = pd.DataFrame(pd.date_range(start=start_date_time, periods=(60/time_resolution)*24,
/var/folders/5g/f459qk8s5ql55568q_c08sxr0000gn/T/ipykernel_16325/3678814824.py:14: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  timeseries = pd.DataFrame(pd.date_range(start=start_date_time, periods=(60/time_resolution)*24,


(['2019-01-01 18:00:00'],
 ['2019-01-01 16:00:00'],
                  DEPTIME             ARRTIME
 4072 2019-01-01 14:00:00 2019-01-01 18:00:00
 7090 2019-01-01 08:00:00 2019-01-01 16:00:00,
 [<__main__.EVData at 0x10bca5400>, <__main__.EVData at 0x10c4a5a30>])

In [13]:
file_name = 'ev.pkl'
file_path = os.path.join(ev_model_path, file_name)
print(file_path)

/Users/noahdimonti/Developer/PycharmProjects/EV_model_test/ev.pkl


In [14]:
with open(file_path, 'wb') as f:
    pickle.dump(ev, f)

In [3]:
work_ev = dc.remove_outliers_once(work_ev, 'DEPTIME', 'ARRTIME')

In [4]:
work_ev_1 = dc.create_timeseries_at_home_pattern_once(work_ev, 'DEPTIME', 'ARRTIME', num_of_ev=2, rand_state=0)

In [19]:
work_ev_1.rename(columns={'EV_ID4072': 'EV_1', 'EV_ID7090': 'EV_2'}, inplace=True)
work_ev_1

,timestamp,EV_1,EV_2
0,2019-01-01 00:00:00,1,1
1,2019-01-01 00:15:00,1,1
2,2019-01-01 00:30:00,1,1
3,2019-01-01 00:45:00,1,1
4,2019-01-01 01:00:00,1,1
...,...,...,...
91,2019-01-01 22:45:00,1,1
92,2019-01-01 23:00:00,1,1
93,2019-01-01 23:15:00,1,1
94,2019-01-01 23:30:00,1,1


In [6]:
work_ev_2 = dc.create_timeseries_at_home_pattern_once(work_ev, 'DEPTIME', 'ARRTIME', num_of_ev=2, rand_state=1)

In [20]:
work_ev_2.timestamp = pd.date_range(start='2019-01-02 00:00:00', periods=96, freq='15min')
work_ev_2.rename(columns={'EV_ID5873': 'EV_1', 'EV_ID7108': 'EV_2'}, inplace=True)
work_ev_2

,timestamp,EV_1,EV_2
0,2019-01-02 00:00:00,1,1
1,2019-01-02 00:15:00,1,1
2,2019-01-02 00:30:00,1,1
3,2019-01-02 00:45:00,1,1
4,2019-01-02 01:00:00,1,1
...,...,...,...
91,2019-01-02 22:45:00,1,1
92,2019-01-02 23:00:00,1,1
93,2019-01-02 23:15:00,1,1
94,2019-01-02 23:30:00,1,1


In [23]:
merged = pd.concat([work_ev_1, work_ev_2])
merged.set_index('timestamp', inplace=True)
merged

,EV_1,EV_2
timestamp,,
2019-01-01 00:00:00,1,1
2019-01-01 00:15:00,1,1
2019-01-01 00:30:00,1,1
2019-01-01 00:45:00,1,1
2019-01-01 01:00:00,1,1
...,...,...
2019-01-02 22:45:00,1,1
2019-01-02 23:00:00,1,1
2019-01-02 23:15:00,1,1


In [24]:
merged.to_csv('/Users/noahdimonti/Developer/PycharmProjects/EV_model_test/ev_profile_2_days.csv')

In [29]:
work_ev_sample = work_ev[work_ev.columns[:3]]
work_ev_sample.set_index('timestamp', inplace=True)
work_ev_sample

,EV_ID4072,EV_ID7090
timestamp,,
2019-01-01 00:00:00,1,1
2019-01-01 00:15:00,1,1
2019-01-01 00:30:00,1,1
2019-01-01 00:45:00,1,1
2019-01-01 01:00:00,1,1
...,...,...
2019-01-01 22:45:00,1,1
2019-01-01 23:00:00,1,1
2019-01-01 23:15:00,1,1


In [30]:
work_ev_sample.to_csv('/Users/noahdimonti/Developer/PycharmProjects/EV_model_test/ev_profile.csv')